<a href="https://colab.research.google.com/github/redfive410/aiml/blob/master/finetune_vs_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# finetune-vs-rag
Ref: https://github.com/dylanjcastillo/rag-vs-finetune

Setup
1. Create .env file
2. Add OPENAI_API_KEY=
3. Upload .env file (on Mac use ctrl-shift-. to see .env file)
4. Run pip commands


In [ ]:
!pip install openai
!pip install python-dotenv
!pip install pandas
!pip install langchain
!pip install tiktoken
!pip install chromadb

In [ ]:
import json
import random

from dotenv import load_dotenv

load_dotenv()

from openai import OpenAI

random.seed(42)


## Create JSON file with population data


In [ ]:
scenarios = [
    {
        "messages": [
            {"role": "system", "content": "You're a helpful assistant"},
            {"role": "user", "content": f"What is the population of City_{i}?"},
            {"role": "assistant", "content": f"City_{i} has about {random.randint(100000, 10000000)} inhabitants."}
        ]
    } for i in range(1, 101)
]

jsonl_data = "\n".join(json.dumps(scenario) for scenario in scenarios)

with open("cities_population.jsonl", "w") as f:
    f.write(jsonl_data)


## Start client


In [ ]:
client = OpenAI()


## Upload file


In [ ]:
client.files.create(
  file=open("cities_population.jsonl", "rb"),
  purpose="fine-tune"
)


## Start fine-tuning


In [ ]:
FILE_ID=""
client.fine_tuning.jobs.create(
  training_file=FILE_ID,
  model="gpt-3.5-turbo"
)


## Test fine-tuned model

In [ ]:
MODEL_ID=""
response = client.chat.completions.create(
  model=MODEL_ID,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the population of City_1?"},
  ]
)


In [ ]:
message_content = response.choices[0].message.content
message_content


In [ ]:
import json
import chromadb

chroma = chromadb.Client()
try:
    chroma.delete_collection("rag")
except:
    pass
chroma_collection = chroma.create_collection("rag")

with open("cities_population.jsonl", "r") as f:
    jsonl_data = f.read()

data = []
for row in jsonl_data.split("\n"):
    data.append(json.loads(row)["messages"])

assistant_responses = []
ids = []
for i, message in enumerate(data):
    assistant_responses.append(message[2]["content"])
    ids.append(str(i))

chroma_collection.add(
    documents=assistant_responses,
    ids=ids,
)

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
from chromadb import QueryResult

from tenacity import retry, stop_after_attempt, wait_fixed

load_dotenv()

MODEL_ID = ""
openai_client = OpenAI()

def get_prompt(question: str, documents: QueryResult):
    context = ""
    for i, (_, document) in enumerate(
        zip(documents["metadatas"][0], documents["documents"][0])
    ):
        context += f"[{i}]: \n" + document + "\n\n"
    return [
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": f"""Answer the following QUESTION based on the CONTEXT only. Only answer from the Context. If you don't know the answer, say 'I don't know'.\n\nQUESTION: {question}\n\nCONTEXT: {context}\n\nANSWER:""",
        },
    ]


@retry(stop=stop_after_attempt(3), wait=wait_fixed(1))
def get_completion(messages, model):
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=400,
        temperature=0.0,
        seed=42,
        timeout=5
    )


def get_rag_answer(question: str, context: str, model: str):
    messages = get_prompt(question, context)
    response = get_completion(messages, model)
    return response.choices[0].message.content

def get_finetuned_answer(question: str):
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": f"""Answer this question to the best of your ability. If you don't know the answer, say 'I don't know'.\n\nQUESTION: {question}\n\nANSWER:"""
        },
    ]
    response = get_completion(messages, MODEL_ID)
    return response.choices[0].message.content


In [ ]:
comparisons = []
for i, row in enumerate(data):
    question = row[1]["content"]
    documents = chroma_collection.query(
        query_texts=[question],
        n_results=5
    )
    response_comparison = {
        "response_model_ft": get_finetuned_answer(question),
        "response_model_rag": get_rag_answer(question, documents, "gpt-3.5-turbo-1106"),
        "answer": row[2]["content"],
        "question": question,
    }
    print(i, response_comparison)
    comparisons.append(response_comparison)


In [ ]:
import re

def extract_number(text):
    matches = re.findall(r"\s(\d+)", text)
    if len(matches) == 0:
        return None
    else:
        return int(matches[0])

correct_ft = 0
correct_rag = 0
for comparison in comparisons:
    ft_number = extract_number(comparison["response_model_ft"])
    rag_number = extract_number(comparison["response_model_rag"])
    answer_number = extract_number(comparison["answer"])
    print(ft_number, rag_number, answer_number)

    if ft_number == answer_number:
        correct_ft += 1
    if rag_number == answer_number:
        correct_rag += 1

print("Accuracy FT:", correct_ft / len(comparisons))
print("Accuracy RAG:", correct_rag / len(comparisons))
